# IndianBatsModel - Testing & Inference

This notebook tests the trained Bat Species Classifier on new audio files.

**Prerequisites:**
1.  **Trained Model**: You must have a trained `.pth` model file (e.g., from the training notebook).
2.  **Test Data**: Audio files organized in folders by species (similar to training data).

**Steps:**
1.  Setup Environment (Clone code).
2.  Prepare Test Data (Generate Spectrograms).
3.  Load Model.
4.  Evaluate Accuracy.
5.  Run Inference on individual files.


In [ ]:
# 1. Setup Environment
!git clone https://github.com/Quarkisinproton/IndianBatsModel.git
!pip install librosa pyyaml pandas matplotlib scikit-learn

In [ ]:
# 2. Import Modules
import sys
import os
import torch
import numpy as np
import pandas as pd
from pathlib import Path
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report, confusion_matrix

# Add repo path
REPO_DIR = '/kaggle/working/IndianBatsModel'
if REPO_DIR not in sys.path: sys.path.append(REPO_DIR)

# Import project modules
try:
    from MainShitz.data_prep.generate_annotations import generate_annotations
    from MainShitz.data_prep.wombat_to_spectrograms import process_all as generate_spectrograms
    from MainShitz.data_prep.extract_end_frequency import process_all_and_write_csv as extract_features
    from MainShitz.datasets.spectrogram_with_features_dataset import SpectrogramWithFeaturesDataset
    from MainShitz.models.cnn_with_features import CNNWithFeatures
    print("Imports successful!")
except ImportError as e:
    print(f"Import Error: {e}")
    print("Ensure the repository is cloned and REPO_DIR is correct.")


In [ ]:
# 3. Configuration
WORK_DIR = '/kaggle/working'

# --- INPUTS ---
# Search for the model file in /kaggle/input if it's not in the working directory
# Since you uploaded the model as a dataset, it will be in /kaggle/input/<dataset-name>/...
MODEL_PATH = '/kaggle/working/models/bat_fused_best.pth' # Default fallback

# Try to auto-detect the model file in /kaggle/input
found_models = []
for root, dirs, files in os.walk('/kaggle/input'):
    for file in files:
        if file.endswith('.pth'):
            found_models.append(os.path.join(root, file))

if found_models:
    MODEL_PATH = found_models[0]
    print(f"Auto-detected model at: {MODEL_PATH}")
    if len(found_models) > 1:
        print(f"Warning: Multiple models found: {found_models}. Using the first one.")
else:
    print("No .pth model found in /kaggle/input. Please set MODEL_PATH manually.")

# Path to TEST audio folders
# (You can use the same folders as training to verify, or new folders for testing)
TEST_AUDIO_DIRS = [
    '/kaggle/input/pip-ceylonicusbat-species',
    '/kaggle/input/pip-tenuisbat-species'
]

# --- OUTPUTS ---
TEST_JSON_DIR = os.path.join(WORK_DIR, 'test_data/annotations')
TEST_SPECT_DIR = os.path.join(WORK_DIR, 'test_data/spectrograms')
TEST_FEATURES_DIR = os.path.join(WORK_DIR, 'test_data/features')
TEST_FEATURES_CSV = os.path.join(TEST_FEATURES_DIR, 'test_features.csv')

# Ensure directories exist
Path(TEST_FEATURES_DIR).mkdir(parents=True, exist_ok=True)

print(f"Model Path: {MODEL_PATH}")
print(f"Test Data Output: {TEST_SPECT_DIR}")

In [ ]:
# 4. Prepare Test Data
# We need to convert the raw test audio into spectrograms and features, just like training.

print("--- Step 1: Generating Annotations ---")
generate_annotations(
    raw_audio_dirs=TEST_AUDIO_DIRS,
    output_dir=TEST_JSON_DIR,
    label_strategy='folder'
)

print("\n--- Step 2: Generating Spectrograms ---")
generate_spectrograms(
    raw_audio_dirs=TEST_AUDIO_DIRS,
    json_dir=TEST_JSON_DIR,
    out_dir=TEST_SPECT_DIR,
    species_key='label'
)

print("\n--- Step 3: Extracting Features ---")
extract_features(
    raw_audio_dirs=TEST_AUDIO_DIRS,
    json_dir=TEST_JSON_DIR,
    out_csv=TEST_FEATURES_CSV,
    species_key='label'
)
print("\nTest Data Preparation Complete.")

In [ ]:
# 5. Load Test Dataset
try:
    test_dataset = SpectrogramWithFeaturesDataset(
        root_dir=TEST_SPECT_DIR,
        features_csv=TEST_FEATURES_CSV
    )
    print(f"Loaded Test Dataset: {len(test_dataset)} samples")
    print(f"Classes: {test_dataset.class_to_idx}")
except Exception as e:
    print(f"Error loading dataset: {e}")
    print("Did spectrogram generation fail?")
    test_dataset = []

In [ ]:
# 6. Load Trained Model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

if len(test_dataset) > 0:
    # Infer dimensions from dataset
    sample_img, sample_feat, _ = test_dataset[0]
    FEAT_DIM = sample_feat.shape[0]
    
    # Try to load class mapping from model path
    class_map_path = MODEL_PATH + '.classes.json'
    if os.path.exists(class_map_path):
        import json
        with open(class_map_path, 'r') as f:
            class_map = json.load(f)
        NUM_CLASSES = len(class_map)
        print(f"Loaded {NUM_CLASSES} classes from {class_map_path}")
    else:
        NUM_CLASSES = 3 # Fallback
        print(f"Warning: Class map not found. Using default NUM_CLASSES={NUM_CLASSES}")
    
    print(f"Initializing model with num_classes={NUM_CLASSES}, feat_dim={FEAT_DIM}")
    
    # Initialize a fresh model (in case we load state_dict)
    model = CNNWithFeatures(num_classes=NUM_CLASSES, numeric_feat_dim=FEAT_DIM, pretrained=False)
    
    if os.path.exists(MODEL_PATH):
        try:
            print(f"Loading model from {MODEL_PATH}...")
            # Fix for PyTorch 2.6+ security change: weights_only=False
            checkpoint = torch.load(MODEL_PATH, map_location=device, weights_only=False)
            
            if isinstance(checkpoint, torch.nn.Module):
                print("Detected full model object. Using loaded model directly.")
                model = checkpoint
            elif isinstance(checkpoint, dict):
                print("Detected state_dict.")
                if 'model_state_dict' in checkpoint:
                    model.load_state_dict(checkpoint['model_state_dict'])
                else:
                    model.load_state_dict(checkpoint)
            else:
                print(f"Warning: Unknown checkpoint format: {type(checkpoint)}. Trying to load as state_dict anyway.")
                model.load_state_dict(checkpoint)
                
            model.to(device)
            model.eval()
            print("Model loaded successfully!")
        except Exception as e:
            print(f"Error loading model: {e}")
            import traceback
            traceback.print_exc()
    else:
        print(f"CRITICAL: Model file not found at {MODEL_PATH}")
        print("Please upload your trained model or check the path.")
else:
    print("Cannot load model: Dataset is empty.")


In [ ]:
# 7. Evaluate Accuracy
if len(test_dataset) > 0 and 'model' in locals():
    test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)
    
    all_preds = []
    all_labels = []
    
    print("Running evaluation...")
    with torch.no_grad():
        for images, features, labels in test_loader:
            images, features = images.to(device), features.to(device)
            outputs = model(images, features)
            _, preds = torch.max(outputs, 1)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.numpy())
    
    # Calculate Metrics
    class_names = list(test_dataset.class_to_idx.keys())
    
    print("\n" + "="*40)
    print("CLASSIFICATION REPORT")
    print("="*40)
    print(classification_report(all_labels, all_preds, target_names=class_names))
    
    print("\nCONFUSION MATRIX:")
    print(confusion_matrix(all_labels, all_preds))
else:
    print("Skipping evaluation (missing model or data).")

In [ ]:
# 8. Detailed Inference on Random Samples
import pandas as pd
import random
from IPython.display import display

# Number of samples to check
NUM_SAMPLES = 15

if len(test_dataset) > 0 and 'model' in locals():
    # Select random indices
    indices = random.sample(range(len(test_dataset)), min(NUM_SAMPLES, len(test_dataset)))
    
    results = []
    correct_count = 0
    
    model.eval()
    idx_to_class = {v: k for k, v in test_dataset.class_to_idx.items()}
    
    print(f"Testing CNN on {len(indices)} random samples...")
    
    with torch.no_grad():
        for idx in indices:
            img, feat, label = test_dataset[idx]
            
            # Get filename (spectrogram file)
            # Accessing the internal image_paths list from the dataset
            full_path = test_dataset.image_paths[idx]
            filename = os.path.basename(full_path)
            
            # Prepare batch
            img_batch = img.unsqueeze(0).to(device)
            feat_batch = feat.unsqueeze(0).to(device)
            
            # Inference
            outputs = model(img_batch, feat_batch)
            probs = torch.nn.functional.softmax(outputs, dim=1)
            conf, pred_idx = torch.max(probs, 1)
            
            pred_class = idx_to_class[pred_idx.item()]
            true_class = idx_to_class[label.item()]
            confidence = conf.item() * 100
            
            is_correct = (pred_class == true_class)
            if is_correct:
                correct_count += 1
                
            results.append({
                'filename': filename,
                'expected': true_class,
                'predicted': pred_class,
                'confidence': f"{confidence:.1f}%",
                'correct': is_correct
            })
    
    # Calculate Accuracy
    accuracy = (correct_count / len(indices)) * 100
    print(f"\nCNN Accuracy (on this random subset): {accuracy:.1f}%")
    
    # Create and display DataFrame
    df_results = pd.DataFrame(results)
    display(df_results)
    
else:
    print("Skipping inference demo.")